##Повторить чтение файлов со своими файлами со своей схемой.
##Создать свой топик/топики, загрузить туда через консоль осмысленные данные с kaggle. Лучше в формате json. Много сообщений не нужно, достаточно штук 10-100.
##Прочитать свой топик так же, как на уроке.


In [1]:
!pip install kaggle

In [2]:
!mkdir ~/.kaggle

In [3]:
!cp kaggle.json ~/.kaggle/

In [4]:
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle datasets download joebeachcapital/gpa-and-iq

  0% 0.00/795 [00:00<?, ?B/s]
100% 795/795 [00:00<00:00, 2.27MB/s]


In [6]:
! unzip /content/gpa-and-iq

Archive:  /content/gpa-and-iq.zip
  inflating: gpa_iq.csv              


In [7]:
import pandas as pd
df = pd.read_csv('gpa_iq.csv')
df

,obs,gpa,iq,gender,concept
0,1,7.940,111,2,67
1,2,8.292,107,2,43
2,3,4.643,100,2,52
3,4,7.470,107,2,66
4,5,8.882,114,1,58
...,...,...,...,...,...
73,85,9.000,112,1,60
74,86,9.500,112,1,70
75,87,6.057,114,2,51
76,88,6.057,93,1,21


In [8]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=0cfe3e77c12b6ccf2fdc2a85fed83ea8d6843767db8794618500122b281a3316
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [9]:
!curl -sSOL https://dlcdn.apache.org/kafka/3.5.0/kafka_2.13-3.5.0.tgz

!tar -xzf kafka_2.13-3.5.0.tgz

In [10]:
!./kafka_2.13-3.5.0/bin/zookeeper-server-start.sh -daemon ./kafka_2.13-3.5.0/config/zookeeper.properties
!./kafka_2.13-3.5.0/bin/kafka-server-start.sh -daemon ./kafka_2.13-3.5.0/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10

Waiting for 10 secs until kafka and zookeeper services are up and running


In [11]:
!ps -ef | grep kafka

root        1256       1 23 06:24 ?        00:00:02 java -Xmx512M -Xms512M -server -XX:+UseG1GC -XX:MaxGCPauseMillis=20 -XX:InitiatingHeapOccupancyPercent=35 -XX:+ExplicitGCInvokesConcurrent -XX:MaxInlineLevel=15 -Djava.awt.headless=true -Xlog:gc*:file=/content/kafka_2.13-3.5.0/bin/../logs/zookeeper-gc.log:time,tags:filecount=10,filesize=100M -Dcom.sun.management.jmxremote -Dcom.sun.management.jmxremote.authenticate=false -Dcom.sun.management.jmxremote.ssl=false -Dkafka.logs.dir=/content/kafka_2.13-3.5.0/bin/../logs -Dlog4j.configuration=file:./kafka_2.13-3.5.0/bin/../config/log4j.properties -cp /content/kafka_2.13-3.5.0/bin/../libs/activation-1.1.1.jar:/content/kafka_2.13-3.5.0/bin/../libs/aopalliance-repackaged-2.6.1.jar:/content/kafka_2.13-3.5.0/bin/../libs/argparse4j-0.7.0.jar:/content/kafka_2.13-3.5.0/bin/../libs/audience-annotations-0.13.0.jar:/content/kafka_2.13-3.5.0/bin/../libs/commons-cli-1.4.jar:/content/kafka_2.13-3.5.0/bin/../libs/commons-lang3-3.8.1.jar:/content/kafka_2.1

In [12]:
!./kafka_2.13-3.5.0/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 2 --topic gpa

Created topic gpa.


In [13]:
!./kafka_2.13-3.5.0/bin/kafka-topics.sh  --list --bootstrap-server localhost:9092

gpa


In [14]:
import csv
import json

csvfile = open('gpa_iq.csv', 'r')
jsonfile = open('gpa_iq.json', 'w')


reader = csv.DictReader(csvfile)
for row in reader:
   json.dump(row, jsonfile)
   jsonfile.write('\n')

jsonfile = open('gpa_iq.json', 'w')

In [15]:
!./kafka_2.13-3.5.0/bin/kafka-console-producer.sh --broker-list localhost:9092 --topic gpa < gpa_iq.json

In [16]:
!./kafka_2.13-3.5.0/bin/kafka-console-consumer.sh --topic gpa  --bootstrap-server localhost:9092 --from-beginning  --max-messages 5

{"obs": "1", "gpa": "7.94", "iq": "111", "gender": "2", "concept": "67"}
{"obs": "2", "gpa": "8.292", "iq": "107", "gender": "2", "concept": "43"}
{"obs": "3", "gpa": "4.643", "iq": "100", "gender": "2", "concept": "52"}
{"obs": "4", "gpa": "7.47", "iq": "107", "gender": "2", "concept": "66"}
{"obs": "5", "gpa": "8.882", "iq": "114", "gender": "1", "concept": "58"}
Processed a total of 5 messages


In [17]:
!export SPARK_KAFKA_VERSION=0.10
!pyspark --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,org.apache.kafka:kafka-clients:3.3.1

Python 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-359ca99f-85f7-42b0-afe4-8f46090c0b97;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 i

In [18]:
# from pyspark.sql import functions as F
# from pyspark.sql.types import StructType, StringType

# kafka_brokers = "localhost:9092"

# #функция, чтобы выводить на консоль вместо show()
# def console_output(df, freq):
#     return df.writeStream \
#         .format("console") \
#         .trigger(processingTime='%s seconds' % freq ) \
#         .options(truncate=True) \
#         .start()

# #читаем без стрима
# raw_orders = spark.read. \
#     format("kafka"). \
#     option("kafka.bootstrap.servers", kafka_brokers). \
#     option("subscribe", "gpa"). \
#     option("startingOffsets", "earliest"). \
#     load()

# raw_orders.show()

In [19]:
# raw_orders.printSchema()

In [20]:
# raw_orders = spark.readStream. \
#     format("kafka"). \
#     option("kafka.bootstrap.servers", kafka_brokers). \
#     option("subscribe", "gpa"). \
#     option("startingOffsets", "earliest"). \
#     option("maxOffsetsPerTrigger", "5"). \
#     load()

# out = console_output(raw_orders, 5)

In [21]:
# out.stop()

In [22]:
# schema = StructType() \
#     .add("obs", StringType()) \
#     .add("gpa", StringType()) \
#     .add("iq", StringType()) \
#     .add("gender", StringType()) \
#     .add("concept", StringType())


# value_orders = raw_orders \
#     .select(F.from_json(F.col("value").cast("String"), schema).alias("value"), "offset")

# value_orders.printSchema()

In [23]:
# parsed_orders = value_orders.select("value.*", "offset")

# parsed_orders.printSchema()

In [24]:
# out = console_output(parsed_orders, 5)

In [25]:
# out.stop()

In [26]:
# def console_output_ckeckpointed(df, freq):
#      return  df.writeStream \
#          .format("console") \
#          .trigger(processingTime=f'{freq} seconds') \
#          .option("truncate", False) \
#          .option("checkpointLocation", "gpa_checkpoint") \
#          .start()

In [27]:
# out = console_output_ckeckpointed(parsed_orders, 5)

In [28]:
# out.stop()

In [29]:
# exit()